In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cleverhans
%matplotlib inline
from cleverhans.model import Model
import math

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import sgld_tf

Instructions for updating:
Use the retry module or similar alternatives.


/home/henripal/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
class CNN(Model):
    def __init__(self):
        self.layer_names = ['reshape',
                           'conv1',
                           'pool1',
                           'conv2',
                           'pool2',
                           'pool2_flat',
                           'dense',
                           'logits',
                           'probs']
    
    def fprop(self, x):
        reshaped_input = tf.reshape(x, [-1, 28, 28, 1])
        conv1, conv1_pre, conv1_out = self.make_conv_layer(32, [5, 5], reshaped_input)   
        pool1 = tf.layers.max_pooling2d(inputs=conv1_out, pool_size=[2, 2], strides=2)

        conv2, conv2_pre, conv2_out = self.make_conv_layer(64, [5, 5], pool1)         
        pool2 = tf.layers.max_pooling2d(inputs=conv2_out, pool_size=[2, 2], strides=2)
        pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])                              

        dense, dense_pre, dense_out = self.make_dense_layer(1024, pool2_flat)         
        dense2, logits, dense2_out = self.make_dense_layer(10, dense_out)             
        probs = tf.nn.softmax(logits)
        
        return {'reshape': reshaped_input,
               'conv1': conv1_out,
               'pool1': pool1,
               'conv2': conv2_out,
               'pool2': pool2,
               'pool2_flat': pool2_flat,
               'dense': dense_out,
               'logits': logits,
               'probs': probs}
        
    def make_conv_layer(self, n_filters, kernel_size, inputs, padding="same"):
        conv = tf.layers.Conv2D(filters=n_filters,                             
                              kernel_size=kernel_size,                         
                              padding=padding)                                 
        pre = conv(inputs)                                                     
        out = tf.nn.relu(pre)                                                  

        return conv, pre, out                                                  

    def make_dense_layer(self, n_units, inputs):                               
        dense = tf.layers.Dense(units=n_units)                                 
        pre = dense(inputs)                                                    
        out = tf.nn.relu(pre)                                                  

        return dense, pre, out                                                 

In [6]:
from cleverhans.utils_mnist import data_mnist

In [7]:
X_train, Y_train, X_test, Y_test = data_mnist(train_start=0,
                                                  train_end=60000,
                                                  test_start=0,
                                                  test_end=10000)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
X_train shape: (60000, 28, 28, 1)
X_test shape: (10000, 28, 28, 1)


In [8]:
x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
y = tf.placeholder(tf.float32, shape=(None, 10))

In [9]:
model = CNN()

In [10]:
model_dict = model.fprop(x)

In [11]:
preds = model_dict['probs']

In [12]:
logits = model_dict['logits']

In [13]:
loss = tf.losses.softmax_cross_entropy(y, logits)

In [14]:
optimizer = sgld_tf.SGLD(learning_rate = 0.01, lrdecay=1)

In [15]:
global_step = tf.Variable(0, name='global_step', trainable=False)
train_step = optimizer.minimize(loss, global_step=global_step)

In [16]:
saver = tf.train.Saver()

In [17]:
## saver.save(sess, path)
correct_preds = tf.equal(tf.argmax(y, axis=-1), tf.argmax(preds, axis=-1))

In [18]:
batch_size = 512

In [19]:
sess = tf.Session()
with sess.as_default():
    tf.global_variables_initializer().run()
    for epoch in range(4):
        print('epoch: ', epoch)
        nb_batches = int(math.ceil(float(len(X_train)) / batch_size))
        for batch in range(nb_batches):
            start = batch*batch_size
            end = min((batch+1)*batch_size, len(X_train))
            
            feed_dict = {x: X_train[start:end],
                        y: Y_train[start:end]}
            train_step.run(feed_dict=feed_dict)
            myloss = loss.eval(feed_dict=feed_dict)
            print("batch loss: ", myloss)
            
        nb_batches_eval = int(math.ceil(float(len(X_test)) / batch_size))
        
        accuracy = 0.0
        for batch in range(nb_batches_eval):
            start = batch*batch_size
            end = min((batch+1)*batch_size, len(X_test))
            
            
            feed_dict2 = {x: X_train[start:end],
                        y: Y_train[start:end]}
            curr_corr_preds = correct_preds.eval(feed_dict=feed_dict2)
            accuracy += curr_corr_preds.sum()
            
        print("eval_acc ", accuracy/len(X_test))


epoch:  0
batch loss:  2.31034
batch loss:  2.31022
batch loss:  2.30413
batch loss:  2.30159
batch loss:  2.29983
batch loss:  2.30118
batch loss:  2.30262
batch loss:  2.30622
batch loss:  2.29781
batch loss:  2.29579
batch loss:  2.29489
batch loss:  2.29645
batch loss:  2.28804
batch loss:  2.29556
batch loss:  2.29499
batch loss:  2.2899
batch loss:  2.29166
batch loss:  2.29178
batch loss:  2.29473
batch loss:  2.28787
batch loss:  2.28719
batch loss:  2.28379
batch loss:  2.2829
batch loss:  2.29052
batch loss:  2.28562
batch loss:  2.28314
batch loss:  2.28163
batch loss:  2.28169
batch loss:  2.27426
batch loss:  2.27569
batch loss:  2.27667
batch loss:  2.27864
batch loss:  2.27216
batch loss:  2.27349
batch loss:  2.27486
batch loss:  2.26946
batch loss:  2.27399
batch loss:  2.27372
batch loss:  2.27455
batch loss:  2.26534
batch loss:  2.26401
batch loss:  2.26727
batch loss:  2.26679
batch loss:  2.2628
batch loss:  2.2609
batch loss:  2.25583
batch loss:  2.26434
batch l

batch loss:  0.591536
batch loss:  0.590125
batch loss:  0.673926
batch loss:  0.562992
batch loss:  0.63586
batch loss:  0.644825
batch loss:  0.724752
batch loss:  0.548931
batch loss:  0.593006
batch loss:  0.615074
batch loss:  0.657803
batch loss:  0.651391
batch loss:  0.557015
batch loss:  0.541693
batch loss:  0.605047
batch loss:  0.606394
batch loss:  0.721519
batch loss:  0.667961
batch loss:  0.663782
batch loss:  0.703548
batch loss:  0.648112
batch loss:  0.70779
batch loss:  0.597324
batch loss:  0.579956
batch loss:  0.503678
batch loss:  0.548331
batch loss:  0.621853
batch loss:  0.554443
batch loss:  0.558105
batch loss:  0.480322
batch loss:  0.582428
batch loss:  0.680288
batch loss:  0.562503
batch loss:  0.578213
batch loss:  0.465844
batch loss:  0.664197
batch loss:  0.566448
batch loss:  0.535415
batch loss:  0.57708
batch loss:  0.630064
batch loss:  0.628629
batch loss:  0.563072
batch loss:  0.586873
batch loss:  0.51222
batch loss:  0.625457
batch loss:  0

In [20]:
start

9728

In [21]:
end

10000